Outlier Study
========
Here we'll first read the csv file containing the top 37 pair of coins as per hourly trades & hence we'll get a list of those pairs. 

In [1]:
import pandas as pd
import json
import datetime

In [2]:
fp = open("input_csv/ed_trade_data.json")
TradeDb = json.load(fp)

In [3]:
def DataManipulation(df_in):
    ''' This function is for converting object type columns to numeric type columns of the dataframe which is 
    passed as the argument here and returning the required dataframe with all numeric columns. ''' 
    
    df_in['datetime'] = pd.to_datetime(df_in['date'])
    df_in['price'] = df_in['price'].astype(float)
    df_in['amount'] = df_in['amount'].astype(float)
    df_in['amountBase'] = df_in['amountBase'].astype(float)
    required_dataframe = df_in[['datetime', 'amount', 'amountBase', 'price']].set_index('datetime')
    return required_dataframe

In [4]:
def DetectOutlier(input_df, col_name):
    ''' This function is taking a dataframe & one of its column as arguments and then calculating the 
    1st and 3rd quartiles. iqr is the variable where it's calculating the Inter Quartile Range and hence 
    it's calculating the upper and lower bound and checking  whether all the values of that particular
    column belongs withing that boundary. df_out is returning the values which are out of the range. '''
    
    q1 = input_df[col_name].quantile(0.25)
    q3 = input_df[col_name].quantile(0.75)
    iqr = (q3 - q1) #Interquartile range
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    df_out = input_df[(input_df[col_name] < lower_bound) | (input_df[col_name] > upper_bound)]
    return df_out

In [5]:
def ProcessPairlist(data, pair_name):
    ''' This function is taking a dictionary and one of the top traded pair name as arguments and then calling
    another function DataManipulation to make the readymade dataframe and then calling another function 
    DetectOutliers that takes the resultant dataframe as the argument of the previous function to detect 
    the outliers in that dataframe. '''
    
    TradeDataframe = pd.DataFrame(data)
    manipulated_data = DataManipulation(TradeDataframe)
    outliers = DetectOutlier(manipulated_data, 'price')
    outliers.to_csv('output_files/outlier_study/OutlierReport_'+ pair_name +'.csv')
    summary = [pair_name] + PrepareSummary(TradeDataframe, outliers)
    return summary

In [7]:
def PrepareSummary(df1, df2):
    ''' This function is determining the percentage of the outliers. For that it's first determining the shape
    of df1 and the df2 which are passed as arguments in this. Then it's determining amount of data of those two
    dataframes. Then it's calculating the relative percentage of the outliers in the dataframe for each pair of 
    coins and reurning a list of the summary. '''
   
    row1, col1 = df1.shape
    row2, col2 = df2.shape
    relative_percentage = (float(row2)/float(row1)) * 100
    summary = [row1, row2, relative_percentage]
    return summary

In [8]:
def ReportMaking(summary_list, old_report):
    ''' This function is taking the list of summary of outliers as argument and producting a well formated 
    Dataframe containing summary of Outliers.'''
    
    report = pd.DataFrame(summary_list).sort_values(by = 2)
    report.columns = ['pair', 'total_data', 'outliers_in_data', '%_of_outliers']
    initial_report = old_report[['pair', 'min_date', 'max_date', 'total_time', 'no_of_trades', 'avg_hourly_trades', 'volume_traded', 'total_turnover', 'avg_price_of_unit_volume']]
    summary_report = pd.merge(initial_report, report, how = 'left', left_on = 'pair', right_on = 'pair')
    final_report = summary_report.set_index('pair').sort_values(by = '%_of_outliers')
    return final_report

In [10]:
top_hourly_traded_pairs = pd.read_csv('output_files/initial_study/top_hourly_traded_pairs.csv')
PairList = list(top_hourly_traded_pairs['pair'])
info_outliers = []
for pair in PairList:
    
    TradeData = TradeDb[pair]
    SummaryOutliers = ProcessPairlist(TradeData, pair)
    info_outliers.append(SummaryOutliers)

SummaryReport = ReportMaking(info_outliers, top_hourly_traded_pairs)
SummaryReport

,min_date,max_date,total_time,no_of_trades,avg_hourly_trades,volume_traded,total_turnover,avg_price_of_unit_volume,total_data,outliers_in_data,%_of_outliers
pair,,,,,,,,,,,
ETH_0x4dc3,2017-10-30 21:30:44,2017-11-02 16:31:18,"2 days, 19:00:34",500,7.461635,1.467670e+06,944.269962,6.759606e-04,500,2,0.4
ETH_REQ,2017-10-31 09:59:08,2017-11-02 11:16:04,"2 days, 1:16:56",500,10.145646,5.998714e+06,746.949024,1.265370e-04,500,2,0.4
ETH_BTCM,2017-10-30 14:24:59,2017-11-02 14:24:29,"2 days, 23:59:30",500,6.945248,3.759531e+06,85.938696,2.245526e-05,500,3,0.6
ETH_ETG,2017-10-29 20:37:00,2017-11-02 12:46:59,"3 days, 16:09:59",500,5.671095,6.219580e+05,97.236903,1.477370e-04,500,4,0.8
ETH_COSS,2017-10-28 15:05:33,2017-11-02 10:10:08,"4 days, 19:04:35",500,4.344940,4.063773e+06,1161.451789,2.805023e-04,500,4,0.8
ETH_IBTC,2017-11-01 14:12:59,2017-11-02 15:48:44,"1 day, 1:35:45",500,19.534429,2.011444e+06,95.760957,4.760807e-05,500,4,0.8
ETH_KNC,2017-10-28 16:45:34,2017-11-02 11:33:36,"4 days, 18:48:02",500,4.355380,2.099102e+05,701.128377,3.317293e-03,500,4,0.8
ETH_EVX,2017-10-29 11:39:40,2017-11-02 14:50:11,"4 days, 3:10:31",500,5.041579,9.376436e+04,577.679886,5.738842e-03,500,4,0.8
ETH_KICK,2017-10-30 18:11:24,2017-11-02 12:59:07,"2 days, 18:47:43",500,7.485559,2.599542e+06,347.411906,1.376200e-04,500,4,0.8
